## Deep Learning with Keras

The goal here is to try Deep Learning to see if the accuracy of the image classification improves, as Deep Learning and CNNs 
tend to perform well for complex data such as image.

The suggested architecture is the following :
1. Input Layer (3 channel image input layer)
* Convolutional (2D)
* Max Pooling
* Convolutional (2D)
* Max Pooling
* Dense (Output Layer)

In [1]:
import os
import pandas as pd
import numpy as np
np.random.seed(42)
from PIL import Image
from scipy.stats import randint

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten # Core layers used for neural networks
from keras.layers import Convolution2D, MaxPooling2D # CNN layers used for image processing
from keras.utils import np_utils # Tools to transform the data
from keras.utils import normalize

Using TensorFlow backend.


In [2]:
def label_encoder(key):
    label_mapping = {
        "axes" : "1",
        "boots" : "2",
        "carabiners" : "3",
        "crampons" : "4",
        "gloves" : "5",
        "hardshell_jackets" : "6",
        "harnesses" : "7",
        "helmets" : "8",
        "insulated_jackets" : "9",
        "pulleys" : "10",
        "rope" : "11",
        "tents" : "12",
    }
    return int(label_mapping[key])

In [3]:
rootDir = 'gear_images/'
directories = ['axes', 'boots', 'carabiners', 'crampons', 'gloves', 'hardshell_jackets', 'harnesses',
              'helmets', 'insulated_jackets', 'pulleys', 'rope', 'tents']


df = pd.DataFrame()
category = [] # List of labels (numbered [1,12])
pixel_array = [] # List of Flatten Pixel array for each image

for directory in directories:   
    folderPath = rootDir + '/' + directory + '/'
    print('Folder: {}'.format(folderPath))
    for fname in os.listdir(folderPath):
        if fname.endswith('resized_equalized.jpeg'):
            im = Image.open(folderPath + fname)
            im_array = np.array(im, dtype=float)
            
            # Append data to list
            category.append(label_encoder(directory))
            pixel_array.append(im_array)

pd_dict = {
    'pixel_array' : pixel_array,
    'category' : category,
}
df = pd.DataFrame(pd_dict)
df['category'] = pd.to_numeric(df['category'])
df = shuffle(df)

Folder: gear_images//axes/
Folder: gear_images//boots/
Folder: gear_images//carabiners/
Folder: gear_images//crampons/
Folder: gear_images//gloves/
Folder: gear_images//hardshell_jackets/
Folder: gear_images//harnesses/
Folder: gear_images//helmets/
Folder: gear_images//insulated_jackets/
Folder: gear_images//pulleys/
Folder: gear_images//rope/
Folder: gear_images//tents/


In [4]:
print(df.shape)
df.info()
df.head()

(2063, 2)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2063 entries, 1304 to 860
Data columns (total 2 columns):
pixel_array    2063 non-null object
category       2063 non-null int64
dtypes: int64(1), object(1)
memory usage: 48.4+ KB


,pixel_array,category
1304,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...",7
2028,"[[[255.0, 255.0, 255.0], [255.0, 255.0, 255.0]...",12
693,"[[[255.0, 255.0, 255.0], [255.0, 255.0, 255.0]...",5
1788,"[[[255.0, 255.0, 255.0], [255.0, 255.0, 255.0]...",11
29,"[[[28.0, 29.0, 33.0], [2.0, 3.0, 7.0], [35.0, ...",1


### Spit into Training and Testing Datasets

In [5]:
# Splitting Training and Testing Data
X_train, X_test, y_train, y_test = train_test_split(df['pixel_array'].values.tolist(),
                                                    df['category'].values.tolist(),
                                                    test_size = 0.2,
                                                    random_state=42,
                                                    stratify=df['category'])
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [6]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1650, 128, 128, 3)
(413, 128, 128, 3)
(1650,)
(413,)


### Preprocess input data for Keras

In [7]:
# Transform our dataset from having shape (width, height, depth) to (depth, width, height)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[3], X_train.shape[1], X_train.shape[2])
X_test = X_test.reshape(X_test.shape[0], X_test.shape[3], X_test.shape[1], X_test.shape[2])

In [8]:
print(X_train.shape)
print(X_test.shape)

(1650, 3, 128, 128)
(413, 3, 128, 128)


In [9]:
# convert our data type to float32 and normalize our data values to the range [0, 1].
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = normalize(X_train)
X_test = normalize(X_test)

### Preprocess class labels for Keras

In [10]:
# Convert 1-dimensional class arrays to 12-dimensional class matrices
# For each image, instead of having a scalar value in range(1,12), we now have an array of 0s and 1
Y_train = np_utils.to_categorical(y_train, 13)[:,1:]
Y_test = np_utils.to_categorical(y_test, 13)[:,1:]
print(Y_train.shape)
# Display the first 10 labels
for i in range(0,10):
    print('image ' + str(i+1) + ': ' + str(Y_train[i]))

(1650, 12)
image 1: [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
image 2: [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
image 3: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
image 4: [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
image 5: [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
image 6: [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
image 7: [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
image 8: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
image 9: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
image 10: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


### Define model architecture

In [11]:
# Instantiate sequential model
model = Sequential()

The input shape parameter should be the shape of 1 sample. Here we have (3, 128, 128) that corresponds to  the (depth, width, height) of each image.

Parameters used: 
* 32 convolution filters
* 1 row in each convolution kernel
* 1 column in each convolution kernel


In [12]:
# First Input Layer
model.add(Convolution2D(32, 1, 1, activation='relu', input_shape=(3,128,128)))

# Adding more layers to our model
model.add(Convolution2D(32, 1, 1, activation='relu'))
model.add(MaxPooling2D(pool_size=(1,1)))
model.add(Dropout(0.25)) #to prevent from overfitting

# Additional layers
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(12, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (1, 1), activation="relu", input_shape=(3, 128, 1...)`
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (1, 1), activation="relu")`
  """


In [13]:
print(model.input_shape)
print(model.output_shape)

(None, 3, 128, 128)
(None, 12)


### Compile model

In [14]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

### Fit model on training data

In [19]:
model.fit(X_train, Y_train, epochs=20)

Epoch 1/20
1650/1650 [==============================] - 3s 2ms/step - loss: 0.1280 - acc: 0.9564
Epoch 2/20
1650/1650 [==============================] - 3s 2ms/step - loss: 0.1405 - acc: 0.9473
Epoch 3/20
1650/1650 [==============================] - 3s 2ms/step - loss: 0.1290 - acc: 0.9545
Epoch 4/20
1650/1650 [==============================] - 3s 2ms/step - loss: 0.1146 - acc: 0.9570
Epoch 5/20
1650/1650 [==============================] - 3s 2ms/step - loss: 0.1128 - acc: 0.9600
Epoch 6/20
1650/1650 [==============================] - 3s 2ms/step - loss: 0.1106 - acc: 0.9624
Epoch 7/20
1650/1650 [==============================] - 3s 2ms/step - loss: 0.0819 - acc: 0.9758
Epoch 8/20
1650/1650 [==============================] - 3s 2ms/step - loss: 0.1165 - acc: 0.9576
Epoch 9/20
1650/1650 [==============================] - 3s 2ms/step - loss: 0.1244 - acc: 0.9527
Epoch 10/20
1650/1650 [==============================] - 3s 2ms/step - loss: 0.1069 - acc: 0.9618
Epoch 11/20
1650/1650 [======

### Evaluate model on test data

In [20]:
val_loss, val_acc = model.evaluate(X_test, Y_test)
print(model.metrics_names)
print(val_loss, val_acc)

413/413 [==============================] - 0s 400us/step
['loss', 'acc']
0.5104563689405058 0.8837772386991949


### Save model to Disk

In [ ]:
model.save('gear_classifier.model')